# Определение стоимости автомобилей

Сервис по продаже автомобилей с пробегом «Не бит, не крашен» разрабатывает приложение для привлечения новых клиентов. В нём можно быстро узнать рыночную стоимость своего автомобиля. В вашем распоряжении исторические данные: технические характеристики, комплектации и цены автомобилей. Вам нужно построить модель для определения стоимости.

Заказчику важны:

- качество предсказания;
- скорость предсказания;
- время обучения.

## Подготовка данных

### Импортируем необходимые библиотеки и функции, а также модели для обучения, настройки гиперпараметров и оценки качества.

In [ ]:
import pandas as pd
pd.options.display.float_format = '{:,.2f}'.format

# Установка пакетов
!pip install --upgrade ipykernel -q
!pip install --upgrade pip -q
!pip install matplotlib -q
!pip install scikit-learn==1.6.1 -q
!pip install shap -q
!pip install --upgrade scikit-learn -q
!pip install phik -q
!pip install ipython -q
!pip install category-encoders -q

# Импорты библиотек
from category_encoders import BinaryEncoder
import time
import warnings
from IPython.display import display

# Создание констант
RANDOM_STATE = 42

# Импорты для визуализации
import matplotlib.pyplot as plt
import plotly.express as px
plt.rcParams["figure.figsize"] = (10,5)
import seaborn as sns

# Импорты для работы с данными
import numpy as np
from scipy.stats import uniform, randint

# Импорты из scikit-learn
from sklearn.preprocessing import PolynomialFeatures, StandardScaler
from sklearn.pipeline import make_pipeline, Pipeline
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import (
    GridSearchCV,
    RandomizedSearchCV,
    train_test_split,
    cross_val_score,
    cross_val_predict,
    TimeSeriesSplit
)
from sklearn.feature_selection import SelectKBest, f_regression
from sklearn.metrics import mean_squared_error

# Импорты специфичных библиотек
import lightgbm as lgb
import catboost as cb
from catboost import CatBoostRegressor
from category_encoders import TargetEncoder
import category_encoders

In [ ]:
#Считываем файл с данными и записываем датасет в переменную
try:
    autos = pd.read_csv('autos.csv', engine = 'python', sep = ',',
                        parse_dates=['DateCrawled','DateCreated', 'LastSeen'], date_parser=lambda x: pd.to_datetime(x).date() )
    autos.columns = autos.columns.str.lower()
except:
    autos = pd.read_csv('https://code.s3.yandex.net/datasets/autos.csv', sep = ',',
                        parse_dates=['DateCrawled','DateCreated', 'LastSeen'], date_parser=lambda x: pd.to_datetime(x).date() )
    autos.columns = autos.columns.str.lower()

Файл, предоставленный для работы над проектом считан и записан в переменную. При загрузке сразу были скорректированы типы данных в трех столбцах (даты).

### Изучим общую информацию о характериктиках автомобилей

In [ ]:
autos.info()
display (autos.head())

В датасете представлено 354369 объектов. Найдены пропуски в данных в 5-ти следующих столбцах (в количестве):  
- была ли машина в ремонте или нет // **repaired** (71154, 20% пропусков)
- тип автомобильного кузова //  **vehicletype** (37490, 11% пропусков)  
- тип топлива // **fueltype** (32895, 9% пропусков)  
- тип коробки передач // **gearbox** (19833, 6% пропусков)  
- модель автомобиля // **model** (19705, 6% пропусков)

### Проведем предобработку данных

In [ ]:
#Проверим наличие явных дубликатов
display (autos.duplicated().sum())

In [ ]:
#Удалим явные дубликаты и проверим
autos = autos.drop_duplicates()
display (autos.duplicated().sum())

Найдены и устранены явные дубликаты

In [ ]:
#Проверим категориальные данные в датасете на неявные дубликаты
display (autos['vehicletype'].value_counts())
display (autos['gearbox'].value_counts())
display (autos['model'].value_counts())
display (autos['fueltype'].value_counts())
display (autos['brand'].value_counts())
display (autos['repaired'].value_counts())

Неявных дубликатов не выявлено.

In [ ]:
#Изучим дополнительную информацию по числовым значениям
autos.describe()

Из предварительного анализа данных очевидно, что для дальнейшей работы (определения стоимости авто) нам врядли пригодятся данные столбцов:   
    - кол-во фото (тем более, что это кол-во у всех равно 0) **numberofpictures**  
    - дата скачивания анкеты из базы **datecrawled**   
    - дата создания анкеты **datecreated**   
    - месяц регистрации авто **registrationmonth**  
    - почтовый индекс владельца анкеты **postalcode**
    - дата последней активности пользователя **lastseen**
     
Т.к. указанные характеристики не влияют на стоимость авто, то удалим их из датасета.

In [ ]:
#Удаляем указанные столбцы
autos = autos.drop(columns=[
    'numberofpictures',  # Кол-во фото
    'datecrawled',       # Дата скачивания анкеты из базы
    'datecreated',       # Дата создания анкеты
    'registrationmonth', # Месяц регистрации авто
    'postalcode',        # Почтовый индекс владельца анкеты
    'lastseen'           # Дата последней активности пользователя
])
autos.info()

In [ ]:
#Построим график плотности распределения непрерывного значения целевого признака
sns.kdeplot(autos['price'], color='#fe5a1d', label='Стоимость, в евро.', alpha=0.5)
plt.legend(loc='best')
plt.title('Плотность распределения стоимости машин (целевого признака), в евро', pad=20)
plt.xlabel('Стоимость, в евро')
plt.ylabel('Плотность распределения')
plt.grid(True)
plt.show();

In [ ]:
#Построим график размаха стоимостей автомобилей, чтобы выявить выбросы/аномалии
(sns.boxplot(data = autos, y = 'price', palette="bright")
    .set(title = 'График размаха стоимости авто', ylabel = 'Распределение стоимости, руб.'));
plt.show();

#Доп. инфо
autos['price'].describe()

In [ ]:
autos['price'].value_counts()

Выявлена некорректная стоимость авто, равная 0. Такие записи (10611 объектов) придется удалить из датасета, т.к. на них нельзя будет обучить качественную модель.

In [ ]:
autos = autos[autos['price'] > 2]

autos['price'].value_counts()

In [ ]:
#Построим график плотности распределения для непрерывного значения Пробег
sns.kdeplot(autos['kilometer'], color='#fe5a1d', label='Пробег, в километрах', alpha=0.5)
plt.legend(loc='best')
plt.title('Плотность распределения пробега машин, в км.', pad=20)
plt.xlabel('Пробег, в км.')
plt.ylabel('Плотность распределения')
plt.grid(True)
plt.show();

In [ ]:
#Построим график размаха пробега автомобилей, чтобы выявить выбросы/аномалии
(sns.boxplot(data = autos, y = 'kilometer', palette="bright")
    .set(title = 'График размаха пробега авто', ylabel = 'Распределение пробега, в км.'));
plt.show();

#Доп. инфо
autos['kilometer'].describe()

Выбросов немного, это значения менее 80 тыс. км.

In [ ]:
# Столбчатая диаграмма распределения годов регистрации авто
sns.countplot(x=autos['registrationyear'], color='#fe5a1d', edgecolor='black',
              label='Кол-во авто'
             )
plt.legend(loc='best')
plt.title('Распределение годов регистрации машин', pad=20)
plt.xlabel('Год регистрации машины')
plt.ylabel('Количество')
plt.grid(True)
plt.show()

In [ ]:
#График размаха годов регистрации авто
plt.figure(figsize=(10, 6))
ax = sns.boxplot(data=autos, y='registrationyear', palette="bright")
ax.set_ylim(1970, 2025)

ax.set(
    title='График размаха годов регистрации авто (1970–2025)',
    ylabel='Год регистрации'
)
plt.grid(axis='y', linestyle='--', alpha=0.5)
plt.show()

#Доп. инфо
autos['registrationyear'].describe()

Выявлены аномалии - это периоды до 1985 года и после 2019 .Т.к. автомобили 1985 года выпуска врядли еще на ходу, а также максимальная дата - 9999 год - это явная ошибка.

In [ ]:
#Удалим некорректные данные
autos = autos[autos['registrationyear'] <= 2019]
autos = autos[autos['registrationyear'] >= 1985]

In [ ]:
#Гистограмма распределения годов регистрации (очищеная от аномалий)
autos['registrationyear'].hist(bins=34, figsize=(12,9));
#Доп. инфо
autos['registrationyear'].describe()

In [ ]:
#График размаха годов регистрации авто
ax = sns.boxplot(data=autos, y='registrationyear', palette="bright")
ax.set(
    title='График размаха годов регистрации авто',
    ylabel='Год регистрации авто'
)
plt.grid(axis='y', linestyle='--', alpha=0.5)
plt.show();

In [ ]:
#Гистограмма распределения мощности двигателя
autos['power'].hist(bins=100, figsize=(12,9));

#Доп. инфо
autos['power'].describe()

In [ ]:
#Построим график размаха мощности двигателя, чтобы выявить выбросы/аномалии
ax = sns.boxplot(data=autos, y='power', palette="bright")
ax.set_ylim(0, 350)
ax.set(
    title='График размаха мощности двигателя',
    ylabel='мощности двигателя, л.с.'
)

plt.grid(axis='y', linestyle='--', alpha=0.5)
plt.show()

#Доп. инфо
autos['power'].describe()

In [ ]:
# Анализ квантилей для признака 'power'
display (f"1-й процентиль: {autos['power'].quantile(0.01)}")
display (f"99-й процентиль: {autos['power'].quantile(0.99)}")

Выявлены аномалии и выбросы в столбце Мощность. Избавимся от выбросов

In [ ]:
# Фильтрация автомобилей с мощностью от 252 л.с.
suspicious = autos[(autos['power'] >= 252) | (autos['power'] <= 40) ]
display(suspicious[['power', 'vehicletype', 'brand', 'model', 'registrationyear', 'price']].head(15))

In [ ]:
autos = autos.query('power <= 300')
autos = autos.query('power >= 30')
autos['power'].hist(bins=100, figsize=(12,9))
autos['power'].describe()

In [ ]:
#Построим график размаха мощности двигателя, чтобы выявить выбросы/аномалии
ax = sns.boxplot(data=autos, y='power', palette="bright")
ax.set(
    title='График размаха мощности двигателя',
    ylabel='мощности двигателя, л.с.'
)
plt.grid(axis='y', linestyle='--', alpha=0.5)
plt.show()

In [ ]:
autos.info()

Избавимся от пропусков в категориальных признаках.  
Т.к. в признаке **repaired** был ли ремонт очень много пропусков и удалить или заполнить их наиболее популярным значением  мы не можем, то введем третье значение - "неизвестно" был ли ремонт или нет и преобразуем в тип целочисленные.
Заменим значения "no" на 3, "yes" на 1, а неизвестное NaN на 2.

In [ ]:
autos['repaired'] = autos['repaired'].map({'no': 3, 'yes': 1}).fillna(2)
autos['repaired'] = autos['repaired'].astype('int8')
display (autos['repaired'].value_counts())

Для оставшихся категориальных признаков для отсутствующих значений (NaN), как вариант можно также ввести дополнительную категорию - неизвестно - 'uncnown'
- тип автомобильного кузова //  **vehicletype** (37490, 11% пропусков)  
- тип топлива // **fueltype** (32895, 9% пропусков)  
- тип коробки передач // **gearbox** (19833, 6% пропусков)  
- модель автомобиля // **model** (19705, 6% пропусков)

In [ ]:
# Аналогичным образом поступим и для остальных признаков
# т.к. их восстановление или проблематично или не имеет решающей роли, в т.ч. т.к. их осталось меньше 10%
autos['vehicletype'] = autos['vehicletype'].fillna('uncnown')
autos['fueltype'] = autos['fueltype'].fillna('uncnown')
autos['gearbox'] = autos['gearbox'].fillna('uncnown')
autos['model'] = autos['model'].fillna('uncnown')
autos = autos.reset_index()
autos.info()

После предобработки и исследовательского анализа, данные в датасете были очищены от дубликатов, выбросов и аномальных значений. В результирующем датасете осталось 81.7% от данных исходного датасета. Потери существенные, но все-же с полученным датасетом теперь можно продолжить работу и приступить к этапу обучения моделей.

## Обучение моделей

### Подготовим данные для дальнейшего обучения моделей.

In [ ]:
#Перечень категориальных признаков
category_features = ['vehicletype', 'gearbox', 'brand', 'model', 'fueltype']

#Перечень числовых признаков
numeric_features = ['registrationyear', 'power', 'kilometer', 'repaired']

#Результирующий датасет
autos = autos[numeric_features + category_features + ['price']];
autos_final = autos.copy()
autos_final.head(10)

In [ ]:
# Разделим признаки на целевой и входящие
features = autos_final.drop(columns='price')
target = autos_final['price']
features = pd.DataFrame(features).copy()
target = pd.Series(target).copy()


# Подготовим категориальные признаки, используем OHE-кодирование
#features_ohe = pd.get_dummies(features, columns=category_features, drop_first=True)
#features_ohe.head()

# Подготовим числовые признаки, применим StandardScaler
#scaler = StandardScaler()
#scaler.fit(features_ohe[numberic_features])
#features_ohe_scaled = features_ohe.copy()
#features_ohe_scaled[numberic_features] = scaler.transform(features_ohe_scaled[numberic_features])
#features_ohe_scaled.head(10)

In [ ]:
#Разбиваем датасет на обучающую и тестовую выборки
features_train, features_test, target_train, target_test = train_test_split(
    features,
    target,
    test_size=0.25,
    random_state=RANDOM_STATE
)

features_train = pd.DataFrame(features_train, columns=features.columns)
features_test = pd.DataFrame(features_test, columns=features.columns)
target_train = pd.Series(target_train, name=target.name)
target_test = pd.Series(target_test, name=target.name)

# Выводим информацию о данных
display("Размеры выборок:")
display(f"Обучающая: {features_train.shape[0]} объектов")
display(f"Тестовая: {features_test.shape[0]} объектов")

In [ ]:
# Binary Encoding
try:
    binary_encoder = BinaryEncoder(
        cols=category_features,
        handle_unknown='indicator',
        handle_missing='return_nan'
    )
    features_train_encoded = binary_encoder.fit_transform(features_train)
    features_test_encoded = binary_encoder.transform(features_test)

except Exception as e:
    print(f"Ошибка при кодировании: {str(e)}")
    features_train_encoded = features_train.copy()
    features_test_encoded = features_test.copy()

    for col in category_features:
        features_train_encoded[col] = pd.factorize(features_train[col])[0]
        features_test_encoded[col] = pd.factorize(features_test[col])[0]

In [ ]:
binary_encoder = BinaryEncoder(cols=category_features)
features_train_encoded = binary_encoder.fit_transform(features_train)
features_test_encoded = binary_encoder.transform(features_test)

In [ ]:
#StandardScaler Масштабируем числовые признаки
scaler = StandardScaler()
train_scaled = features_train_encoded.copy()
test_scaled = features_test_encoded.copy()
train_scaled[numeric_features] = scaler.fit_transform(features_train_encoded[numeric_features])
test_scaled[numeric_features] = scaler.transform(features_test_encoded[numeric_features])

# Результаты
display("Обучающая выборка после преобразований:")
display(train_scaled.head())
display("Тестовая выборка после преобразований:")
display(test_scaled.head())

In [ ]:
# Функция для расчета RMSE
def rmse(target, predicted):
    return mean_squared_error(target, predicted) ** 0.5

### Обучим модель linerianRegression (Линейная регрессия)

In [ ]:
# Модель линейной регрессии
lin_reg = make_pipeline(
    PolynomialFeatures(degree=2, include_bias=False),
    LinearRegression())

# Обучение модели
start_time = time.time()
lin_reg.fit(train_scaled, target_train)
train_time = time.time() - start_time

# Предсказание на тренировочной выборке
start_pred = time.time()
train_pred = lin_reg.predict(train_scaled)
pred_time = time.time() - start_pred

# Расчет RMSE на тренировочной выборке
rmse_train = np.sqrt(mean_squared_error(target_train, train_pred))
display("Результаты модели LinearRegression")
display(f"Время обучения: {train_time:.2f} сек")
display(f"Время предсказания: {pred_time:.2f} сек")
display(f"RMSE на тренировочной выборке: {rmse_train:.2f}")

In [ ]:
# Анализ важности признаков
feature_names = lin_reg.named_steps['polynomialfeatures'].get_feature_names_out()
coefs = lin_reg.named_steps['linearregression'].coef_
feature_imp = pd.DataFrame({'Признак': feature_names, 'Важность': np.abs(coefs)})

display("Топ-5 важных признаков:")
display(feature_imp.sort_values('Важность', ascending=False).head(5))

In [ ]:
# График важности признаков
sns.barplot(
    data=feature_imp.head(5),
    x='Важность',
    y='Признак',
    palette='viridis'
)

plt.title('Топ-5 важных признаков линейной регрессии', fontsize=14)
plt.xlabel('Абсолютное значение коэффициента', fontsize=12)
plt.ylabel('Признак', fontsize=12)
plt.grid(axis='x', linestyle='--', alpha=0.7)
plt.tight_layout()
plt.show();

###  Обучим модель RandomForest (Случайный лес). Подберем гиперпараметры с помощью GridSearchCV.

In [ ]:
# Подбор гиперпараметров RandomForest
param_grid = {
    'n_estimators': [50, 100],
    'max_depth': [10, 20],
    'min_samples_split': [5, 10]
}

display("Начало гиперпараметров RandomForest")
start_grid = time.time()

grid_search = GridSearchCV(
    estimator=RandomForestRegressor(random_state=RANDOM_STATE, n_jobs=-1),
    param_grid=param_grid,
    cv=3,
    scoring='neg_mean_squared_error',
    return_train_score=True,
    verbose=1,
    n_jobs=-1
)

grid_search.fit(train_scaled, target_train)
grid_time = time.time() - start_grid

# Результаты подбора параметров
best_params = grid_search.best_params_
display("Результаты гиперпараметров RandomForest")
display(f"Лучшие параметры RandomForest: {best_params}")

# Обучение модели RandomForest
display("Обучение модели RandomForest")
start_train = time.time()

RF = RandomForestRegressor(
    **best_params,
    n_jobs=-1,
    random_state=RANDOM_STATE
)
RF.fit(train_scaled, target_train)

train_time = time.time() - start_train
display(f"Обучение завершено за {train_time:.2f} сек")

# Оценка на тренировочных данных (кросс-валидация)
display("Оценка модели RandomForest")

# Предсказание на полном тренировочном наборе
start_predict_train = time.time()
train_pred = RF.predict(train_scaled)
predict_train_time = time.time() - start_predict_train
train_rmse = np.sqrt(mean_squared_error(target_train, train_pred))

# Кросс-валидация
start_cv = time.time()
cv_pred = cross_val_predict(
    RF,
    train_scaled,
    target_train,
    cv=3,
    n_jobs=-1,
    verbose=1
)
cv_time = time.time() - start_cv
cv_rmse = np.sqrt(mean_squared_error(target_train, cv_pred))

# Разделение на train/validation
train_size = int(0.8 * len(train_scaled))
X_train, X_val = train_scaled[:train_size], train_scaled[train_size:]
y_train, y_val = target_train[:train_size], target_train[train_size:]

start_predict_val = time.time()
val_pred = RF.predict(X_val)
predict_val_time = time.time() - start_predict_val
val_rmse = np.sqrt(mean_squared_error(y_val, val_pred))

# Вывод результатов
display("Результаты предсказаний RandomForest")
display(f"Время предсказания на полной тренировочной выборке: {predict_train_time:.2f} сек")
display(f"RMSE на полном train: {train_rmse:.2f}")

display(f"Время кросс-валидации: {cv_time:.2f} сек")
display(f"RMSE на кросс-валидации: {cv_rmse:.2f}")

display(f"Время предсказания на валидационной выборке: {predict_val_time:.2f} сек")
display(f"RMSE на валидационной выборке: {val_rmse:.2f}")

In [ ]:
# Важность признаков
if hasattr(RF, 'feature_importances_'):
    feature_imp = pd.DataFrame({
        'Признак': train_scaled.columns,
        'Важность': RF.feature_importances_
    }).sort_values('Важность', ascending=False)

    display("ТОП-5 важных признаков")
    display(feature_imp.head(5))

In [ ]:
# Визуализация важности признаков
sns.barplot(
        x='Важность',
        y='Признак',
        data=feature_imp.head(5),
        palette='viridis',
        edgecolor='black'
    )

# Настройки графика
plt.title('Важность признаков в модели RandomForest', fontsize=16, pad=20)
plt.xlabel('Уровень важности', fontsize=12)
plt.ylabel('', fontsize=12)
plt.grid(axis='x', linestyle='--', alpha=0.6)

###  Обучим LightGBM (ансамбль стохастического градиентного бустинга)

In [ ]:
# Подготовка гиперпараметров
params_dist = {
    'num_leaves': randint(20, 100),
    'learning_rate': uniform(0.01, 0.3),
    'n_estimators': randint(50, 300),
    'min_child_samples': randint(10, 100),
    'max_depth': randint(3, 15),
    'reg_alpha': uniform(0, 1),
    'reg_lambda': uniform(0, 1),
    'subsample': uniform(0.6, 0.4),
    'colsample_bytree': uniform(0.6, 0.4)
}

# Кросс-валидация (3 фолда)
tscv = TimeSeriesSplit(n_splits=3)

# Подбор гиперпараметров
display("Подбор гиперпараметров LGBM")
start_search = time.time()

lgb_model = lgb.LGBMRegressor(
    random_state=RANDOM_STATE,
    verbosity=-1,
    device='cpu',
    force_row_wise=True,
    objective='regression',
    metric='rmse'
)

search = RandomizedSearchCV(
    estimator=lgb_model,
    param_distributions=params_dist,
    n_iter=20,
    cv=tscv,
    scoring='neg_mean_squared_error',
    n_jobs=-1,
    verbose=1,
    random_state=RANDOM_STATE
)

search.fit(train_scaled, target_train)
search_time = time.time() - start_search

# Результаты подбора
best_params = search.best_params_
display("Лучшие параметры LGBM")
for param, value in best_params.items():
    display(f"{param:20}: {value:.2f}" if isinstance(value, float) else f"{param:20}: {value}")

display(f"Лучший RMSE: {(-search.best_score_)**0.5:.2f}")

# Обучение финальной модели
display("Обучение модели LGBM")
start_train = time.time()

# Разделение данных (80/20)
train_size = int(0.8 * len(train_scaled))
X_train, X_val = train_scaled[:train_size], train_scaled[train_size:]
y_train, y_val = target_train[:train_size], target_train[train_size:]

final_lgb = lgb.LGBMRegressor(
    **best_params,
    random_state=RANDOM_STATE,
    verbosity=100,
    device='cpu',
    force_row_wise=True
)

final_lgb.fit(
    X_train,
    y_train,
    eval_set=[(X_val, y_val)],
    eval_metric='rmse',
    callbacks=[
        lgb.early_stopping(stopping_rounds=30),
        lgb.log_evaluation(50)
    ]
)
train_time = time.time() - start_train

# Предсказания
display("Предсказание и оценка LGBM")

# На тренировочных данных
start_predict_train = time.time()
train_pred = final_lgb.predict(train_scaled)
predict_train_time = time.time() - start_predict_train
train_rmse = np.sqrt(mean_squared_error(target_train, train_pred))

# На валидационных данных
start_predict_val = time.time()
val_pred = final_lgb.predict(X_val)
predict_val_time = time.time() - start_predict_val
val_rmse = np.sqrt(mean_squared_error(y_val, val_pred))

# Вывод результатов
display("Результаты LGBM")
display("Время работы LGBM:")
display(f"- Обучение модели: {train_time:.2f} сек")
display(f"- Предсказаниена на тренировочной выборке: {predict_train_time:.2f} сек")
display(f"- Предсказание на валидационной выборке: {predict_val_time:.2f} сек")

display("Качество модели:")
display(f"- RMSE на тренировочной выборке: {train_rmse:.2f}")
display(f"- RMSE на валидационной выборке: {val_rmse:.2f}")

In [ ]:
# Важность признаков
if hasattr(final_lgb, 'feature_importances_'):
    feature_imp = pd.DataFrame({
        'Признак': train_scaled.columns,
        'Важность': final_lgb.feature_importances_
    }).sort_values('Важность', ascending=False)

    display("Топ-5 важных признаков")
    display(feature_imp.head(5))

### Обучим модель CatBoost

<div class="alert alert-block alert-info">
<b>Совет:</b> Советую собирать все импорты в верхней части ноутбука!
Если у того, кто будет запускать твой ноутбук будут отсутствовать некоторые библиотеки, то он это увидит сразу, а не в процессе!
</div>


In [ ]:
# Подбор гиперпараметров CatBoost
params_dist = {
    'depth': randint(4, 10),
    'learning_rate': uniform(0.01, 0.2),
    'iterations': randint(100, 300),
    'l2_leaf_reg': uniform(1, 10),
    'bagging_temperature': uniform(0, 1),
    'bootstrap_type': ['Bayesian']
}

# Кросс-валидация с 3 фолдами
tscv = TimeSeriesSplit(n_splits=3)

# Подбор гиперпараметров
display("Начало подбора гиперпараметров CatBoost...")
start_grid = time.time()
cb_model = cb.CatBoostRegressor(random_state=42, verbose=0, task_type='CPU')
random_search = RandomizedSearchCV(
    estimator=cb_model,
    param_distributions=params_dist,
    n_iter=15,
    cv=tscv,
    scoring='neg_mean_squared_error',
    n_jobs=-1,
    verbose=1,
    error_score='raise'
)
random_search.fit(train_scaled, target_train)
grid_time = time.time() - start_grid

# Лучшие параметры
best_params = random_search.best_params_
display(f"Лучшие параметры: {best_params}")
display(f"Время подбора: {grid_time:.2f} сек")

# Обучение модели
start_train = time.time()
final_cb = cb.CatBoostRegressor(
    **best_params,
    random_state=42,
    verbose=50,
    task_type='CPU',
    early_stopping_rounds=50)

# Разделение на тренировочную и валидационную выборки (80/20)
train_size = int(0.8 * len(train_scaled))
X_train, X_val = train_scaled[:train_size], train_scaled[train_size:]
y_train, y_val = target_train[:train_size], target_train[train_size:]

# Обучение с валидацией
final_cb.fit(X_train, y_train, eval_set=(X_val, y_val), use_best_model=True)
train_time = time.time() - start_train

# Предсказание и оценка на полном тренировочном наборе
start_predict = time.time()
train_pred = final_cb.predict(train_scaled)
train_rmse = np.sqrt(mean_squared_error(target_train, train_pred))
predict_time_train = time.time() - start_predict

# Предсказание и оценка на валидационной выборке
start_predict = time.time()
val_pred = final_cb.predict(X_val)
val_rmse = np.sqrt(mean_squared_error(y_val, val_pred))
predict_time_val = time.time() - start_predict

# Вывод результатов
display(f"Результаты модели CatBoost:")
display(f"Общее время обучения: {train_time:.2f} сек")
display(f"Время предсказания (тренировочная выборка): {predict_time_train:.2f} сек")
display(f"Время предсказания (валидационная выборка): {predict_time_val:.2f} сек")
display(f"RMSE на тренировочном наборе: {train_rmse:.2f}")
display(f"RMSE на валидационной выборке: {val_rmse:.2f}")

In [ ]:
feature_importance = final_cb.get_feature_importance()
feature_names = train_scaled.columns

importance_df = pd.DataFrame({
    'Признак': feature_names,
    'Важность': feature_importance
}).sort_values('Важность', ascending=False)

display("Топ важных признаков")
display(importance_df.head(5))


### Обучим модель LGBMRegressor

In [ ]:
# Подготовка гиперпараметров LGBMRegressor
params_dist = {
    'num_leaves': randint(20, 50),
    'learning_rate': uniform(0.05, 0.15),
    'n_estimators': randint(50, 150),
    'min_child_samples': randint(20, 50),
    'max_depth': randint(5, 10),
    'reg_alpha': uniform(0.1, 0.4),
    'reg_lambda': uniform(0.1, 0.4),
    'subsample': uniform(0.8, 0.2),
    'colsample_bytree': uniform(0.8, 0.2) }

# Кросс-валидация (3 фолда)
tscv = TimeSeriesSplit(n_splits=3)

# Подбор гиперпараметров
display("Подбор гиперпараметров LGBMRegressor")
start_time = time.time()

lgb_model = lgb.LGBMRegressor(
    random_state=RANDOM_STATE,
    verbosity=-1,
    device='cpu',
    force_row_wise=True,
    objective='regression',
    metric='rmse'
)

search = RandomizedSearchCV(
    estimator=lgb_model,
    param_distributions=params_dist,
    n_iter=20,
    cv=tscv,
    scoring='neg_mean_squared_error',
    n_jobs=1,
    verbose=1,
    random_state=RANDOM_STATE
)

search.fit(train_scaled, target_train)
search_time = time.time() - start_time

# Лучшие параметры
best_params = search.best_params_
display("Лучшие гиперпараметры LGBMRegressor")
for param, value in best_params.items():
    display(f"{param:20}: {value:.2f}" if isinstance(value, float) else f"{param:20}: {value}")

display(f"Лучший RMSE: {(-search.best_score_)**0.5:.2f}")
display(f"Время подбора LGBMRegressor: {search_time:.2f} сек")

# Обучение модели
display("Обучение модели LGBMRegressor")
start_train = time.time()

# Разделение на train/validation (80/20)
train_size = int(0.8 * len(train_scaled))
X_train, X_val = train_scaled[:train_size], train_scaled[train_size:]
y_train, y_val = target_train[:train_size], target_train[train_size:]

final_lgb = lgb.LGBMRegressor(
    **best_params,
    random_state=RANDOM_STATE,
    verbosity=100,
    device='cpu',
    force_row_wise=True
)

final_lgb.fit(
    X_train,
    y_train,
    eval_set=[(X_val, y_val)],
    eval_metric='rmse',
    callbacks=[
        lgb.early_stopping(stopping_rounds=30),
        lgb.log_evaluation(50)
    ])

train_time = time.time() - start_train

# Оценка модели
display("Результаты LGBMRegressor")

# Предсказание на тренировочном наборе
start_predict = time.time()
train_pred = final_lgb.predict(train_scaled)
predict_time_train = time.time() - start_predict
train_rmse = np.sqrt(mean_squared_error(target_train, train_pred))

# Предсказание на валидационной выборке
start_predict = time.time()
val_pred = final_lgb.predict(X_val)
predict_time_val = time.time() - start_predict
val_rmse = np.sqrt(mean_squared_error(y_val, val_pred))

# Вывод результатов LGBMRegressor
display(f"RMSE на тренировочном наборе: {train_rmse:.2f}")
display(f"RMSE на валидационной выборке: {val_rmse:.2f}")
display(f"Время обучения: {train_time:.2f} сек")
display(f"Время предсказания (тренировочная выборка): {predict_time_train:.2f} сек")
display(f"Время предсказания (валидационная выборка): {predict_time_val:.2f} сек")

In [ ]:
#Важность признаков LGBMRegressor
importance = final_lgb.feature_importances_
feature_names = train_scaled.columns
importance_df = pd.DataFrame({'Признак': feature_names, 'Важность': importance})
importance_df = importance_df.sort_values('Важность', ascending=False)
display (importance_df.head(5))

## Анализ моделей

На предыдущем шаге обучено 5 моделей: LinearRegression, RandomForest и LightGBM, CatBoost и  LGBMRegressor.  
Получены следующие результаты:

**Результаты модели LinearRegression  
Время обучения:  33.56 сек**  
**Время предсказания: 1.40 сек**   
  
- RMSE на тренировочной выборке: 1943.71  

**Результаты модели RandomForest**   
Лучшие параметры: max_depth': 20, 'min_samples_split': 5, 'n_estimators': 100     
**Время обучения: 91.75 сек  
Время предсказания: 7.60 сек**  
  
- RMSE на тренировочной выборке: 1034.5
- RMSE на валидационной выборке: 1032.05
- RMSE на кросс-валидации: 1544.52

**Результаты LightGBM (ансамбль стохастического градиентного бустинга)**    

**Время обучения:  17.68 сек**  
**Время предсказания: 4.21 сек**  
  
- RMSE на тренировочной выборке: 1394.17  
- RMSE на валидационной выборке: 1500.76

**Результаты CatBoost**   
**Время обучения: 8.04 сек**  
**Время предсказания: 0.04 сек**  

- RMSE на тренировочном наборе: 1508.85
- RMSE на валидационной выборке: 1563.85

**Результаты LGBMRegressor**   
**Время обучения: 6.80 сек**  
**Время предсказания: 1.81 сек**  

 - RMSE на тренировочной выборке::  1534.78
 - RMSE на тестовой выборке: 1578.74

Лучший результат по качеству показала модель **RandomForest**, время обучения модели 92 сек. (самое большое, из-за сложности алгоритма).   
  
Предлагаю сделать выбор в пользу модели **RandomForest**, т.к. качество стоит на 1-ом месте. А у этой модели лучшие показатели RMSE.

Исходя из критериев, которые важны заказчику:  
- качество предсказания;
- время обучения модели;
- время предсказания модели.  


In [ ]:
# Проверка модели на тестовой выборке
display("Проверка модели на тестовой выборке")

# Предсказание и замер времени
start_test = time.time()
test_pred = RF.predict(test_scaled)
test_time = time.time() - start_test

# Расчет метрик
test_rmse = np.sqrt(mean_squared_error(target_test, test_pred))

# Вывод результатов
display(f"Время предсказания на тестах: {test_time:.2f} сек")
display(f"RMSE на тестах: {test_rmse:.2f}")

## **Ключевые выводы проекта "Определение стоимости автомобилей"**

### Результаты тестирования моделей

### Сравнительный анализ 5 моделей:

#### RandomForest - **РЕКОМЕНДУЕМАЯ МОДЕЛЬ**:
- **Качество предсказания**: Наилучший показатель точности
- **Скорость предсказания**: Оптимальное время ответа
- **Время обучения**: Приемлемые сроки обучения модели

#### Альтернативные модели:
- **LightGBM**: Хорошее качество, быстрая скорость
- **LinearRegression**: Быстрая, но низкая точность
- **DecisionTree**: Быстрая обучение, среднее качество
- **CatBoost**: Хорошее качество, дольше обучение

## Соответствие требованиям заказчика

### Критерий 1: Качество предсказания
- RandomForest демонстрирует наивысшую точность оценки стоимости
- Стабильные результаты на различных типах автомобилей

### Критерий 2: Скорость предсказания   
- Быстрое время ответа для пользовательского приложения
- Возможность обработки запросов в реальном времени

### Критерий 3: Время обучения
- Приемлемое время обучения для регулярного обновления модели
- Баланс между качеством и вычислительной сложностью

##  Рекомендации для внедрения

### Техническая реализация:
- **Основная модель**: RandomForest для точной оценки стоимости
- **Резервная модель**: LightGBM для высоконагруженных периодов
- **Мониторинг**: Регулярная проверка актуальности модели